In [ ]:
# -*- coding: utf8 -*-
import pyes
import elasticsearch
import json
import numpy.numarray as na
from pylab import *
from matplotlib import pyplot as plt 
import pandas as pd
from bokeh.charts import BoxPlot, output_file, show


'''
我的code是在本機端寫的，
所以是直接複製上ipynb
圖片在ppt內都有
'''
#connect to DB@ES
es_address='127.0.0.1:9200'
conn = pyes.es.ES(es_address)
# 192.168.1.100:9300

month_dict = {0 : 'Jan', 1 : 'Feb', 2 : 'Mar', 3 : 'Apr', 4 : 'May', 5 : 'Jun',
			  6 : 'Jul', 7 : 'Aug', 8 : 'Sep', 9 : 'Oct', 10: 'Nov', 11: 'Dec'}
month_list = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
month_number_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

#### 2a, 2b
#########aggregation + term
bq = pyes.query.BoolQuery() 
tq = pyes.query.TermQuery(field="text", value="exploit")

ESR = pyes.ESRange(field="created_at", from_value="Tue Jan 01 00:00:00 +0000 2014", to_value="Fri Jan 01 00:00:00 +0000 2015", 
                   include_lower=True ,include_upper=True)
rq = pyes.query.RangeQuery(qrange=ESR)

bq.add_must(tq)
bq.add_must(rq)

qsearch = pyes.query.Search(bq)

tagg = pyes.aggs.TermsAgg('uid', field= 'uid')

qsearch.agg.add(tagg)

result = conn.search(query=qsearch , indices='twitter2', doc_types='tweet') 
print len(result.aggs['uid']['buckets'])
print len(result)
print result[0]['text'].encode('utf-8')


In [ ]:
###### 2c, 2d
month_twitters = list()
month_tweets = list()

for i in range(12):
	bq = pyes.query.BoolQuery() 
	tq = pyes.query.TermQuery(field="text", value="exploit")
	bq.add_must(tq)

	from_value = "Tue " + month_dict[i] + " 01 00:00:00 +0000 2014"

	if i == 11:
		to_value = "Fri " + month_dict[0] + " 01 00:00:00 +0000 2015"
	else:
		to_value = "Fri " + month_dict[i+1] + " 01 00:00:00 +0000 2014"
	ESR = pyes.ESRange(field="created_at", from_value=from_value, to_value=to_value, 
	                   include_lower=True ,include_upper=True)
	rq = pyes.query.RangeQuery(qrange=ESR)
	bq.add_must(rq)


	qsearch = pyes.query.Search(bq)

	tagg = pyes.aggs.TermsAgg('uid', field= 'uid')

	qsearch.agg.add(tagg)

	result = conn.search(query=qsearch , indices='twitter2', doc_types='tweet') 

	month_tweets.append(len(result))
	month_twitters.append(len(result.aggs['uid']['buckets']))

print month_twitters
print month_tweets

x1 = na.array(range(12))
x2 = na.array(range(12))
x3 = na.array(range(12))

#draw bar
width = 0.2
b1 = bar(x1, month_tweets, width=width, color = 'c')

#draw tick
yticks(range(0,100,5))
xticks(x1+0.1, month_list)

#label Y axis
ylabel('Y axis', fontsize=12, fontweight='bold')

# xlim(0, 12)
# ylim(0,70)
# title("Number of month_tweets",fontsize=12, fontweight='bold')
title("Number of Exploit month_tweets",fontsize=12, fontweight='bold')
gca().get_xaxis().tick_bottom()
gca().get_yaxis().tick_left()

# savefig('bar_chart.png')
# savefig('bar_chart.pdf')
plt.show()

In [ ]:
#### 2e
bq = pyes.query.BoolQuery() 

tq = pyes.query.TermQuery(field="text", value="exploit")

ESR = pyes.ESRange(field="created_at", from_value="Tue Jan 01 00:00:00 +0000 2014", to_value="Fri Jan 01 00:00:00 +0000 2015", 
                   include_lower=True ,include_upper=True)
rq = pyes.query.RangeQuery(qrange=ESR)

bq.add_must(tq)
bq.add_must(rq)

qsearch = pyes.query.Search(bq)
result = conn.search(query=qsearch , indices='twitter2', doc_types='tweet') 

total_count = len(result)
non_url_count = 0

for temp in result :
	if len(temp['entities']['urls']) == 0:
		non_url_count = non_url_count + 1
print non_url_count
print total_count - non_url_count
# print total_count
# print json.dumps(result[0]['entities']['urls'], indent=2)

labels = ['url','non_url']
sizes = [total_count - non_url_count, non_url_count]
colors = ['red','yellowgreen']

patches,l_text,p_text = plt.pie(sizes,labels=labels,colors=colors,
                                labeldistance = 1.1,autopct = '%3.1f%%',shadow = False,
                                startangle = 90,pctdistance = 0.6)
# print patches
# print l_text
# print p_text
plt.axis('equal')
plt.legend()
plt.show()


In [ ]:

#### 2f
month_twitters = list()
month_tweets = list()

box_data = list()

for i in range(12):
	bq = pyes.query.BoolQuery() 
	tq = pyes.query.TermQuery(field="text", value="exploit")
	bq.add_must(tq)

	from_value = "Tue " + month_list[i] + " 01 00:00:00 +0000 2014"

	if i == 11:
		to_value = "Fri " + month_list[0] + " 01 00:00:00 +0000 2015"
	else:
		to_value = "Fri " + month_list[i+1] + " 01 00:00:00 +0000 2014"

	ESR = pyes.ESRange(field="created_at", from_value=from_value, to_value=to_value, 
	                   include_lower=True ,include_upper=True)
	rq = pyes.query.RangeQuery(qrange=ESR)
	bq.add_must(rq)


	qsearch = pyes.query.Search(bq)

	tagg = pyes.aggs.TermsAgg('uid', field= 'uid')
	qsearch.agg.add(tagg)

	result = conn.search(query=qsearch , indices='twitter2', doc_types='tweet') 

	month_tweets.append(len(result))
	month_twitters.append(len(result.aggs['uid']['buckets']))


	for j in range(len(result.aggs['uid']['buckets'])):
		# print result.aggs['uid']['buckets'][j]['doc_count']
		# box_data.append([result.aggs['uid']['buckets'][j]['doc_count'], month_list[i]])
		box_data.append([result.aggs['uid']['buckets'][j]['doc_count'], month_number_list[i]])
# print month_twitters
# print month_tweets

# print box_data
box_data_pandas = pd.DataFrame(box_data)
box_data_pandas.columns = ['Count', 'Month']

# print box_data_pandas
temp = box_data_pandas.boxplot(by='Month',figsize=(12,8))
temp.set_axis_bgcolor("darkorange")
plt.show()